In [17]:
import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import sys

sys.path.append("../../../")

import os
from pathlib import Path
import itertools

import srsly
import pandas as pd
from statistics import mean

import numpy as np
from unidecode import unidecode
from tqdm.auto import tqdm, trange
import torch
from sentence_transformers.util import cos_sim
from collections import defaultdict
from baselines.kbqa.subgraph_retriever import NHopRetriever, FilteredTriplesNHopRetriever
from gqqd.defaults import ANNOTATED_PATH, OUTPUT_PATH, WIKIDATA_GRAPHS, ROOT_PATH
from gqqd.data.inforex import load_inforex_output_df
from gqqd.data.loaders import load_wiki_content_df
from baselines.kbqa.data import load_dataset, load_graph
from baselines.defaults import KBQA_BASELINES_PATH
from baselines.kbqa.predictor import GPTClient

## Load data

In [19]:
dataset = load_dataset("kbqa_natural")
triples, labels = load_graph(1)
triples = np.array(triples)
df = pd.DataFrame(dataset["train"])
queries_emb = torch.load(
    KBQA_BASELINES_PATH / "embeddings/emb_kbqa_natural_train.pt", map_location="cpu"
)
triples_emb = torch.load(
    KBQA_BASELINES_PATH / "embeddings/emb_triples_hop_1.pt", map_location="cpu"
)

In [20]:
config = srsly.read_yaml(ROOT_PATH / "baselines/kbqa/gpt_config_knowledge.yaml")
print(config["final_message_schema"])

Poniżej znajdują się fakty w postaci trójek grafu wiedzy w formacie (encja, relacja, encja), mające znaczenie do udzielenia odpowiedzi na pytanie.
{triples}
Pytanie: {question} 
Encje które są odpowiedzią: 


In [21]:
client = GPTClient([], config["final_message_schema"], "gpt-3.5-turbo")
retriever = FilteredTriplesNHopRetriever(2, triples, labels=labels, triples_emb=triples_emb)

In [24]:
usage = []
accs = []

for example in df.head(2).itertuples():
    print(example.question)
    topic = example.topic
    subgraph = retriever.retrieve(topic, query_emb=queries_emb[example.Index], k_triples=20)
    subgraph_str = ", ".join(
        f"({labels.get(h, h)}, {labels.get(r, r)}, {labels.get(t, t)})" for h, r, t in subgraph
    )
    subgraph_str = ", ".join(
        f'("{labels.get(h, h)}", "{labels.get(r, r)}", "{labels.get(t, t)}")'
        for h, r, t in subgraph
    )
    res = client.get_response(triples=subgraph_str, question=example.question)
    usage.append(res[1]["usage"])
    answer = example.answer
    acc = mean(1 if entity in res[0] else 0 for entity in answer)
    accs.append(acc)
    print(subgraph_str)
    print(f"Answer: {answer}")
    print(f"Accuracy: {acc}")
    print()
    print(res[0])
    print("=" * 80)
    print()
    # break

gdzie zamieszkują łowiczanie?

Answer: ['Q11750550', 'Q643843']
Accuracy: 0

Łowicz, mieszkańcy Łowicza, województwo łódzkie

w jakim regionie leży poznań?
("Prowincja Poznańska", "kontynent", "Europa"), ("Poznań", "miasto partnerskie", "Region Hanower"), ("Województwo poznańskie", "państwo", "Polska"), ("Poznań", "kontynent", "Europa"), ("Region Hanower", "miasto partnerskie", "Poznań"), ("Poznań", "znajduje się w jednostce administracyjnej", "województwo wielkopolskie"), ("powiat poznański", "państwo", "Polska"), ("województwo wielkopolskie", "podział administracyjny", "Poznań"), ("Wielkie Księstwo Poznańskie", "kontynent", "Europa"), ("Poznań", "", ""), ("Województwo poznańskie", "państwo", "Polska Rzeczpospolita Ludowa"), ("Prowincja Poznańska", "państwo", "Królestwo Prus"), ("województwo poznańskie", "państwo", "II Rzeczpospolita"), ("Województwo poznańskie", "znajduje się w jednostce administracyjnej", "Polska"), ("Prowincja Poznańska", "znajduje się w jednostce administracyjnej"

In [8]:
mean(accs)

0.41

In [9]:
usage_df = pd.DataFrame(usage)
cost = (
    usage_df.prompt_tokens.sum() / 1000 * 0.0005 + usage_df.completion_tokens.sum() / 1000 * 0.0015
)
cost_per_example = cost / len(usage_df)
print(f"3.5: Cost per example: {cost_per_example}")
print(f"3.5: Cost for 1120: {cost_per_example * 1120}")

cost = usage_df.prompt_tokens.sum() / 1000 * 0.01 + usage_df.completion_tokens.sum() / 1000 * 0.03
cost_per_example = cost / len(usage_df)
print(f"gpt-4-0125-preview: Cost per example: {cost_per_example}")
print(f"gpt-4-0125-preview: Cost for 1120: {cost_per_example * 1120}")

cost = usage_df.prompt_tokens.sum() / 1000 * 0.03 + usage_df.completion_tokens.sum() / 1000 * 0.06
cost_per_example = cost / len(usage_df)
print(f"gpt-4: Cost per example: {cost_per_example}")
print(f"gpt-4: Cost for 1120: {cost_per_example * 1120}")

3.5: Cost per example: 0.00025914
3.5: Cost for 1120: 0.29023679999999996
gpt-4-0125-preview: Cost per example: 0.0051827999999999996
gpt-4-0125-preview: Cost for 1120: 5.804735999999999
gpt-4: Cost per example: 0.0151656
gpt-4: Cost for 1120: 16.985471999999998


In [10]:
config = srsly.read_yaml(ROOT_PATH / "baselines/kbqa/gpt_config_noknowledge.yaml")
print(config["final_message_schema"])
client = GPTClient([], config["final_message_schema"], "gpt-3.5-turbo")

Pytanie: {question} 
Encje które są odpowiedzią: 


In [11]:
usage = []
accs = []

for example in df.head(25).itertuples():
    print(example.question)
    res = client.get_response(question=example.question)
    usage.append(res[1]["usage"])
    answer = [labels.get(x["entity_id"]) for x in example.answer]
    acc = mean(1 if entity in res[0] else 0 for entity in answer)
    accs.append(acc)
    print(f"Answer: {answer}")
    print(f"Accuracy: {acc}")
    print()
    print(res[0])
    print("=" * 80)
    print()
    # break

gdzie zamieszkują łowiczanie?
Answer: ['Księstwo łowickie', 'Mazowsze']
Accuracy: 0

Łowiczanie zamieszkują w Łowiczu, mieście położonym w centralnej Polsce, w województwie łódzkim.

w jakim regionie leży poznań?
Answer: ['województwo wielkopolskie']
Accuracy: 0

Wielkopolska

gdzie jest japonia?
Answer: ['Daleki Wschód', 'Ocean Spokojny']
Accuracy: 0

Japonia jest wyspiarskim państwem leżącym na wschodnim wybrzeżu Azji.

kto jest autorem książki „mały książę?
Answer: ['Antoine de Saint-Exupéry']
Accuracy: 1

Antoine de Saint-Exupéry

w jakim powstaniu brał udział wokulski?
Answer: ['powstanie styczniowe']
Accuracy: 0

Wokulski nie brał udziału w żadnym powstaniu.

kim z zawodu był krzysztof komeda?
Answer: ['otorynolaryngologia']
Accuracy: 0

Krzysztof Komeda był polskim muzykiem, kompozytorem i pianistą.

czym jest republika?
Answer: ['ustrój polityczny']
Accuracy: 0

Republika to forma rządów, w której władza jest sprawowana przez przedstawicieli wybranych przez obywateli. W republi

In [12]:
mean(accs)

0.118

In [13]:
usage_df = pd.DataFrame(usage)
cost = (
    usage_df.prompt_tokens.sum() / 1000 * 0.0005 + usage_df.completion_tokens.sum() / 1000 * 0.0015
)
cost_per_example = cost / len(usage_df)
print(f"3.5: Cost per example: {cost_per_example}")
print(f"3.5: Cost for 1120: {cost_per_example * 1120}")

cost = usage_df.prompt_tokens.sum() / 1000 * 0.01 + usage_df.completion_tokens.sum() / 1000 * 0.03
cost_per_example = cost / len(usage_df)
print(f"gpt-4-0125-preview: Cost per example: {cost_per_example}")
print(f"gpt-4-0125-preview: Cost for 1120: {cost_per_example * 1120}")

cost = usage_df.prompt_tokens.sum() / 1000 * 0.03 + usage_df.completion_tokens.sum() / 1000 * 0.06
cost_per_example = cost / len(usage_df)
print(f"gpt-4: Cost per example: {cost_per_example}")
print(f"gpt-4: Cost for 1120: {cost_per_example * 1120}")

3.5: Cost per example: 0.00010446
3.5: Cost for 1120: 0.1169952
gpt-4-0125-preview: Cost per example: 0.0020892
gpt-4-0125-preview: Cost for 1120: 2.3399039999999998
gpt-4: Cost per example: 0.0045192
gpt-4: Cost for 1120: 5.061504
